# 1 스레드와 코루틴 비교

** from Fluent Python **

- 간단한 스레드 프로그램과 그에 준하는 asyncio 버전을 비교하면서, 스레드와 비동기 작업의 관계를 보여준다.
- asyncio.Future 클래스와 concurrent.futures.Future 클래스의 차이점을 설명한다.
- 17장에서 구현한 국기 내려받기 예제의 비동기 버전을 구현한다.
- 스레드나 프로세스를 사용하지 않고 비동기 프로그래밍이 네트워크 프로그램에서 높은 동시성을 관리하는 방법을 설명한다.
- 코루틴으로 비동기 프로그래밍을 하기 위한 콜백을 개선시키는 방법을 설명한다.
- 블로킹 연산을 스레드 풀에 덜어줌으로써 이벤트 루프를 블로킹하지 않는 방법을 알아본다.
- asyncio 서버를 작성하고, 웹 애플리케이션의 높은 동시성을 다시 생각해본다.
- asyncio가 파이썬 생태계에 커다란 영향을 줄 수밖에 없는 이유를 설명한다.

### 스레드와 코루틴 비교

** thread로 구현 **

In [2]:
import threading
import itertools
import time
import sys

class Signal:
    # 외부에서 스레드를 제어하기 위해 사용할 go 속성 하나만 있는 간단한 가변 객체 정의
    go = True
    
def spin(msg, signal):
    # 별도의 스레드에서 실행되는 함수
    # signal 인수는 바로 앞에서 정의한 Signal 클래스의 객체를 받음
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        # itertools.cycle()은 주어진 시퀀스를 순환하면서 끝없이 항목을 생성하므로,
        # 이 for문은 무한 루프가 된다.
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))  # 텍스트 모드 애니메이션 기법. 문자열의 길이만큼 백스페이스를 반복해서 커서를 앞으로 이동
        time.sleep(.1)
        if not signal.go:
            # go 속성이 True가 아니라면 루프를 빠져나온다.
            break
    write(' ' * len(status) + '\x08' * len(status))
    # 공백 문자로 덮어쓰고 다시 커서를 처음으로 이동해서 메시지 출력 행을 청소한다
    
def slow_function():
    # 시간이 오래걸리는 함수라고 가정
    time.sleep(3)
    # 주 스레드에서 sleep() 함수를 호출할 때 GIL이 해제되므로 두번째 스레드가 진행
    return 42

def supervisor():
    # 두번째 스레드를 만들고, 스레드 객체를 출력하고, 시간이 오래 걸리는 연산을 수행 후 스레드 제거
    signal = Signal()
    spinner = threading.Thread(target=spin, args=('thinking!', signal))
    print('spinner object: ', spinner)  # 두번째 스레드 객체 출력
    spinner.start()  # 두번째 스레드 실행
    result = slow_function()  # slow_function() 함수를 실행. 주 스레드 블로킹, 두번째 스레드의 애니메이션 시작
    signal.go = False  # signal의 상태를 변경. spin() 함수 안의 for 루프가 중단 됨
    spinner.join()  # spinner 스레드가 끝날 때까지 기다린다
    return result

def main():
    result = supervisor()  # supervisor() 함수 실행
    print('Answer: ', result)

if __name__ == "__main__":
    main()

spinner object:  | thinking!<Thread(Thread-4, initial)>
/ thinking!- thinking!\ thinking!| thinking!/ thinking!- thinking!\ thinking!| thinking!/ thinking!- thinking!\ thinking!| thinking!/ thinking!- thinking!\ thinking!| thinking!/ thinking!- thinking!\ thinking!| thinking!/ thinking!- thinking!\ thinking!| thinking!/ thinking!- thinking!\ thinking!| thinking!/ thinking!           Answer:  42


**asyncio로 구현**

In [1]:
import asyncio
import itertools
import sys

@asyncio.coroutine  # asyncio에 사용할 코루틴은 @asyncio.coroutine으로 데코레이트 해야함(권장)
def spin(msg):
    # spin() 함수에서 스레드를 종료하기 위해 사용했던 signal 인수가 필요 없음
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write(('\x08') * len(status))
        try:
            yield from asyncio.sleep(.1)
            # 이벤트 루프를 블로킹하지 않고 잠자기 위해 time.sleep(.1) 대신 사용
        except asyncio.CancelledError:
            break
    write(' ' * len(status) + '\x08' * len(status))
    
@asyncio.coroutine
def slow_function():
    # 코루틴이 잠자면서 입출력을 수행하는 체 하는 동안 이벤트 루프가 진행될 수 있게 하기 위해서 yield from을 사용
    # 입출력을 위해 장시간 기다리는 것처럼 보이게 만듬
    yield from asyncio.sleep(3)
    return 42

@asyncio.coroutine
def supervisor():
    # supervisor()도 코루틴이므로, yield from을 이용해서 slow_function()을 구동할 수 있다.
    spinner = asyncio.async(spin('Thinking!'))  # spin() 코루틴의 실행을 스케쥴링하고 Task 객체 않에 넣어, Task 객체를 즉시 반환한다.
    print('spinner object: ', spinner)  # Task 객체 출력
    result = yield from slow_function()  # slow_function() 구동 후 값을 받아온다. 이벤트 루프는 계속 실행 중이다.
    spinner.cancel()  # Task 객체는 cancel() 메소드를 호출해서 취소할 수 있다. 예외를 잡아서 지연시키거나 취소 요청을 거부할 수 있다.
    return result

def main():
    loop = asyncio.get_event_loop()  # 이벤트 루프에 대한 참조 가져오기
    result = loop.run_until_complete(supervisor())
    loop.close()
    print('Answer:', result)
    
if __name__ == '__main__':
    main()

spinner object:  <Task pending coro=<spin() running at <ipython-input-1-63b80426e931>:5>>
| Thinking!/ Thinking!- Thinking!\ Thinking!| Thinking!/ Thinking!- Thinking!\ Thinking!| Thinking!/ Thinking!- Thinking!\ Thinking!| Thinking!/ Thinking!- Thinking!\ Thinking!| Thinking!/ Thinking!- Thinking!\ Thinking!| Thinking!/ Thinking!- Thinking!\ Thinking!| Thinking!/ Thinking!- Thinking!\ Thinking!| Thinking!/ Thinking!           Answer: 42


** _@asyncio.coroutine_ 데코레이터를 사용해야 하는 이유**는, 일반 함수와 다르게 보이도록 만들고, 코루틴이 yield from 되지 않고 가비지 컬렉트가 되는 경우에 경고 메시지를 출력하므로 디버깅에 도움이 된다. 

** _supervisor()_ 함수의 비교**
- asyncio.Task는 threading.Thread와 거의 대등하다. Task는 gevent 같은 협업적 멀티태스킹을 구현하는 라이브러리에서의 green thread와 같다.
- Task는 코루틴을 구동하고, Thread는 콜러블을 호출한다.
- Task 객체는 직접 생성하지 않고, 코루틴을 _asyncio.async()_나 _loop.create_task()_에 전달해서 가져온다.
- Task 객체를 가져오면, 이 객체는 이미 asyncio.async() 등에 의해 실행이 스케쥴링 되어있다. Thread 객체는 _start()_ 메소드를 호출해서 실행하라고 명령해야한다.
- 스레드화된 _supervisor()_에서 _slow_\__function()_은 평범한 함수로서, 스레드에 의해 직접 호출된다. 비동기 _supervisor()_에서 _slow_\__function()_은 yield from으로 구동하는 코루틴이다.
- 스레드는 외부에서 API를 통해 중단시킬 수 없다. 스레드를 아무 때나 중단시키면 시스템 상태의 무결성이 훼손되기 때문이다. Task는 코루틴 안에서 CancelledError를 발생시키는 _Task.cancel()_객체 메소드가 있다. 코루틴은 중단되었던 yield문에서 예외를 잡아서 처리할 수 있다.
- _supervisor()_ 코루틴은 _main()_ 함수 안에서 loop.run_until_complete()로 실행해야 한다.

**스레드와 코루틴의 차이**

스레드의 경우, 스케쥴러가 언제든 스레드를 중단시킬 수 있기 떄문에 분석이 어렵지만, 코루틴의 경우 모든 것이 인터럽트로부터 보호된다. 명시적으로 yield를 실행해야 프로그램의 다른 부분이 실행된다. yield 지점에서만 중단될 수 있기 때문에 다른 코루틴으로 제어권을 넘겨주면 된다.